In [ ]:
# Allows the modification of the files imported without having to restart kernel or re-import them
%load_ext autoreload
%autoreload 2

In [ ]:
%config InlineBackend.figure_format='retina'
from IPython import display
display.display(display.HTML("<style>.container { width:95% !important; }</style>"))

# to have interactive plots : pip install ipympl
%matplotlib ipympl

### General imports
import sys
import glob
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp
import pickle
from scipy.signal import savgol_filter
from scipy.stats import linregress

plt.rc('figure',figsize=(10,6))
plt.rc('font',size=12)

### Astropy configuration
import astropy.units as u

#### QUBIC IMPORT
from qubic.lib import Qdictionary 
from qubic.lib.Instrument import Qinstrument
import qubic.lib.Calibration.Qfiber as ft
import qubicpack.qubicfp as qfp
from qubicpack.plot_fp import plot_fp
# from qubicpack.qubicfp import qubicfp

d = Qdictionary.qubicDict()
dictfilename = '/dicts/global_source_oneDet.dict'
d.read_from_file(dictfilename)
q = Qinstrument.QubicInstrument(d)

# Same for all maps
nside = 256
azqubic = 116.4

### Temporary update of the path 
in order to be able to load libraries that are still in development and not yet in the QUBIC path. This will have to be removed when the relevant libraries are finalized and integrated into QubicSoft

In [ ]:
dirtemplibs = ["/Users/huchet/qubic/qubic/scripts/MoonProject/", "/Users/huchet/Documents/code/scripts/", "/Users/huchet/Documents/code/data/"] #[os.environ['QUBIC_DATADIR']+'scripts/MoonProject/']
for rep in dirtemplibs:     
    if rep not in sys.path:
        sys.path.append(rep)

#### Local files that will need to be installed in the Qubic Libs
import fitting as fit
import time_domain_tools as tdt
import pipeline_moon_plotting as pmp
import pipeline_moon_functions as pmf


## Dataset and Observing site
in the case of the July 2022 Moon observations, they were from Salta CNEA Regional and the UTC offset was -3 hours.

### Location of the raw TOD files
Beware this will have to be changed for each one's configuration

In [ ]:
mydatadir = '/Users/huchet/qubic/qubic/data/Calib-TD/'

### Observation date and corresponding file

In [ ]:
ObsDate = '2025-05-21'
ObsSession = 0
dirs = glob.glob(mydatadir + ObsDate + '/*responsivity')
print(dirs)
datadir = dirs[0]


### Observing Site

In [ ]:
La_Puna = {'lat':-24.186971*u.deg,
            'lon':-66.478209*u.deg,
            'height':4826*u.m,
            'UTC_Offset':-3*u.hour}

Obs_Site = La_Puna

In [ ]:
# try:
#     data_TOD
# except NameError:
print('Reading data from disk: '+datadir)
a = qfp.qubicfp()
a.read_qubicstudio_dataset(datadir)
tt, alltod = a.tod()
az = a.azimuth()
el = a.elevation()
thk = a.timeaxis(datatype='hk')
tinit = tt[0]
# tt, alltod, thk, az, el, tinit = pmf.read_data(datadir, remove_t0=False)
# az += azqubic
# tt_save = np.copy(tt)
# alltod_save = np.copy(alltod)
# thk_save = np.copy(thk)
# az_save = np.copy(az)
# el_save = np.copy(el)
# data_TOD = [tt_save, alltod_save, thk_save, az_save, el_save, tinit]
# print("tinit = {}".format(tinit))

In [ ]:
a.plot_timeline(93,1) # plot the timeline for TES 93 of ASIC 1
a.plot_timeline(93,2) # plot the timeline for TES 93 of ASIC 2

In [ ]:
ASICNum = 2
TESNum = 66 + (ASICNum - 1)*128
tod = alltod[TESNum - 1, :]

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))
ax.plot(tt, tod, label="TOD")
# ax.plot(tt, mytod, label="filtered TOD")
ax.legend()
ax.set_xlabel("Time [s]")
ax.set_ylabel("Flux [ADU]")
plt.tight_layout()
plt.savefig("tod_filtering.pdf")
plt.show()

In [ ]:
def get_deriv(x, y, order): # derivative of TOD, sacrifice order first points for simplicity
    smoothed_y = savgol_filter(y, int(4*order), polyorder=2)
    dy_dx = np.zeros(len(x))
    # dy_dx[:order] = (y[1:order + 1] - y[:order])/(x[1:order + 1] - x[:order])
    # dy_dx[-order:] = (y[-order:] - y[-order - 1:-1])/(x[-order:] - x[-order - 1:-1])
    dt_ = x[2*order:] - x[:-2*order]
    dy_dx[order:-order] = (smoothed_y[2*order:] - smoothed_y[:-2*order])/dt_
    
    plt.figure()
    plt.plot(tt, y, label="original TOD")
    plt.plot(tt, smoothed_y, label="smoothed TOD")
    plt.xlabel("Time [s]")
    plt.ylabel("Flux [ADU]")
    plt.legend()
    plt.show()
    return dy_dx

In [ ]:
def detect_jump(tt, tod, time_interval):
    # time_interval is the time taken in seconds to go from one regime to the other (time needed to put the plate)
    fs = 157 # Hz
    order = int(time_interval*fs)
    print(order, "i,dsg")
    tod_deriv = get_deriv(tt, tod, order=order)

    plt.figure()
    plt.plot(tt, tod_deriv)
    plt.xlabel("Time [s]")
    plt.ylabel("Flux derivative [ADU/s]")
    plt.show()

    threshold = np.std(tod_deriv[tod_deriv**2 < np.median(tod_deriv) + 4 * np.std(tod_deriv)]) # sigma clipping
    flux_jump = np.asarray(tod_deriv**2>threshold**2 * 1000)#.nonzero() # needs to be optimized

    print(np.sum(flux_jump))
    return flux_jump, tod_deriv

In [ ]:
def responsivity_chunk_maker(tt, tod, time_interval=1):
    jump_detection, tod_deriv = detect_jump(tt, tod, time_interval)

    plt.figure()
    plt.plot(tt, tod_deriv)
    plt.plot(tt[jump_detection], tod_deriv[jump_detection])
    plt.show()

    transition = 4*time_interval

    tt_transi_list = [tt[0]]
    tt_transi = tt[0]
    for i in range(np.sum(jump_detection)):
        current_tt = tt[jump_detection][i]
        if current_tt - tt_transi < transition:
            continue

        tt_transi = current_tt
        mask = np.logical_and(tt >= tt_transi, tt < tt_transi + transition)
        tt_transi = tt[mask][np.argmax(tod_deriv[mask]**2)]
        tt_transi_list.append(tt_transi)
    chunks = np.zeros((len(tt_transi_list), len(tt)))
    for i_tt, tt_transi in enumerate(tt_transi_list[:-1]):
        chunks[i_tt] = np.where(np.logical_and(tt >= tt_transi, tt < tt_transi_list[i_tt + 1]), tod, np.zeros(tod.shape))

    return chunks

In [ ]:
chunks = responsivity_chunk_maker(tt, tod, time_interval=1)

In [ ]:
plt.figure()
full_regress = linregress(tt, tod)
plt.plot(tt, full_regress.slope*tt - full_regress.slope*tt[0])
plt.plot(tt, )
plt.show()

In [ ]:
plt.figure()
full_regress = linregress(tt, tod)
for i_chunk, chunk_ in enumerate(chunks):
    tt_chunk = tt[chunk_ != 0]
    chunk = chunk_[chunk_ != 0]
    if tt_chunk.sum() == 0:
        continue
    print(np.max(tt_chunk) - np.min(tt_chunk))
    # chunk_regress = linregress(tt_chunk, chunk)
    chunk_corr = chunk - (full_regress.slope*tt_chunk - full_regress.slope*tt[0])
    plt.plot(tt_chunk, chunk_corr)
plt.show()

In [ ]:
fs = 1/np.median(tt[1:] - tt[:-1]) # Hz # could be computed directly on TOD
print(fs)
lowcut = 1/100 # 4/107.5, i.e. half a forth (or back) scan
tod_filt = pmf.butter_highpass_filter(tod, lowcut=lowcut, fs=fs, order=2) # Hz

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))
ax.plot(tt, tod_filt, label="TOD")
# ax.plot(tt, mytod, label="filtered TOD")
ax.legend()
ax.set_xlabel("Time [s]")
ax.set_ylabel("Flux [ADU]")
plt.tight_layout()
plt.savefig("tod_filtering.pdf")
plt.show()